In [82]:
import json
import numpy as np
import pandas as pd
import os
import sys

from snowflake.snowpark.session import Session
from snowflake.snowpark.functions import sproc, col
import snowflake.snowpark.functions as F
import snowflake.snowpark.types as T

from snowflake.snowpark.types import PandasDataFrameType, IntegerType, StringType, FloatType, Variant
from snowflake.snowpark.exceptions import SnowparkSQLException

In [83]:
# Reading Snowflake Connection Details
snowflake_connection_cfg = json.loads(open("/Users/mitaylor/Documents/creds/creds.json").read())

# Creating Snowpark Session
session = Session.builder.configs(snowflake_connection_cfg).create()

# Create a fresh & new schema
session.sql("CREATE OR REPLACE DATABASE TE_DEMO").collect()
session.sql('''CREATE OR REPLACE STAGE TE_STAGE''').collect()
session.sql("CREATE OR REPLACE WAREHOUSE ASYNC_WH WITH WAREHOUSE_SIZE='X-SMALL'").collect()

[Row(status='Warehouse ASYNC_WH successfully created.')]

In [84]:
from sklearn.datasets import make_classification
import pandas as pd
import numpy as np
import datetime

X,y = make_classification(n_samples=1000, n_features=5, n_classes=2)
X = np.array(X, dtype=np.float32)
df = pd.DataFrame(X)
feature_cols = ["COL" + str(i) for i in df.columns]
df.columns = feature_cols

df['HH'] = np.random.randint(5, size=len(df))
df['GEO'] = [str(i) for i in list(np.random.randint(5, size=len(df)))]
df['CONTRACT_ID'] = np.random.randint(7, size=len(df))
df['ACTUAL'] = y

In [85]:
df

,COL0,COL1,COL2,COL3,COL4,HH,GEO,CONTRACT_ID,ACTUAL
0,0.406002,0.017545,-0.943426,1.218018,1.010194,0,1,6,1
1,-0.620726,0.652450,0.389985,0.131251,-0.312289,3,3,6,0
2,-0.950217,-0.774260,0.172066,1.005823,0.019462,3,0,5,1
3,-1.728694,-1.905334,1.629292,-0.663405,-1.505705,3,3,6,0
4,-2.430774,-0.045657,1.665727,0.251562,-1.385133,3,0,6,0
...,...,...,...,...,...,...,...,...,...
995,-1.952966,0.455013,2.250839,-1.526417,-2.181286,4,4,3,0
996,1.054075,0.332765,-1.189076,0.775039,1.147134,1,1,5,1
997,-1.396142,1.188223,1.640246,-1.150232,-1.595847,4,1,6,0
998,-1.271618,0.337766,0.150457,1.497205,0.119486,3,2,0,1


In [86]:
session.write_pandas(df, table_name='DUMMY_DATASET', auto_create_table=True, overwrite=True)

In [87]:
session.sql('''CREATE OR REPLACE VIEW v1 AS SELECT COL0, COL1, COL2, COL3, COL4, TO_DATE('2018-01-01') AS DATE, HH, GEO, CONTRACT_ID, ACTUAL FROM DUMMY_DATASET;''').collect()

[Row(status='View V1 successfully created.')]

In [88]:
session.table('v1').show()

------------------------------------------------------------------------------------------------------------------------------------------------------------------------
|"COL0"               |"COL1"                |"COL2"               |"COL3"               |"COL4"                |"DATE"      |"HH"  |"GEO"  |"CONTRACT_ID"  |"ACTUAL"  |
------------------------------------------------------------------------------------------------------------------------------------------------------------------------
|0.4060015082359314   |0.017545100301504135  |-0.943425714969635   |1.2180180549621582   |1.0101940631866455    |2018-01-01  |0     |1      |6              |1         |
|-0.6207262873649597  |0.6524500846862793    |0.3899853527545929   |0.13125106692314148  |-0.3122890889644623   |2018-01-01  |3     |3      |6              |0         |
|-0.9502167701721191  |-0.7742597460746765   |0.17206589877605438  |1.0058226585388184   |0.019461724907159805  |2018-01-01  |3     |0      |5             

In [89]:
from snowflake.snowpark.types import PandasDataFrameType, IntegerType, StringType, FloatType, TimeType, DateType

class UDTF_Model_Train:
    def end_partition(self, df_pandas):
        from snowflake.ml.modeling.xgboost import XGBClassifier   

        # When a UDTF ingest a dataframe it does not know what the columns are, you need to tell it (appended with _ to avoid namespace clashes when we come to yield)
        df_pandas.columns = ['COL0_', 'COL1_', 'COL2_', 'COL3_', 'COL4_', 'DATE_', 'HH_', 'GEO_', 'CONTRACT_ID_', 'ACTUAL_']

        # This is generally not necessary but xgboost model can be temperemental with dtypes so it can be helpful to "force" it
        df_pandas = df_pandas.astype({'COL0_': 'float32'})
        df_pandas = df_pandas.astype({'COL1_': 'float32'})
        df_pandas = df_pandas.astype({'COL2_': 'float32'})
        df_pandas = df_pandas.astype({'COL3_': 'float32'})
        df_pandas = df_pandas.astype({'COL4_': 'float32'})
        df_pandas = df_pandas.astype({'ACTUAL_': 'float32'})
        
        try:
            # Define the XGBRegressor using Snowpark ML (exact same library under the hood, but an abstraction to make it easier to work with Snowflake
            final_model = XGBClassifier(
                input_cols=['COL0_', 'COL1_', 'COL2_', 'COL3_', 'COL4_'],
                label_cols=['ACTUAL_'],
                output_cols=['FORECAST_']
            )

            # note you can also run on a snowpark dataframe directly
            final_model.fit(df_pandas)

            # no need to create a predict column, that is handled in the output_cols field above
            df_pandas = final_model.predict(df_pandas) 
            df_print = df_pandas[["DATE_","HH_","ACTUAL_","GEO_","CONTRACT_ID_", "FORECAST_"]]                      
        except Exception as e:
            df_print = pd.DataFrame(columns=["DATE","HH","ACTUAL","GEO","CONTRACT_ID", "FORECAST"])

        #yield statement is per partition, effectively the end result is a stack of all the yield statements alltogether 
        yield df_print

UDTF_Model_Train.end_partition._sf_vectorized_input = pd.DataFrame

udtf_model_inf = session.udtf.register(
    name = "UDTF_MODEL_INF_v1",
    replace=True,
    handler=UDTF_Model_Train, # the class
    input_types=[PandasDataFrameType([FloatType(),FloatType(),FloatType(),FloatType(),FloatType(),
                                      DateType(), IntegerType(), StringType(), IntegerType(), IntegerType()])], 
    output_schema=PandasDataFrameType([DateType(), IntegerType(), FloatType(), StringType(), IntegerType(), FloatType()],
                                      ["DATE","HH","ACTUAL","GEO","CONTRACT_ID", "FORECAST"]), # note these are not appended with _ 
    packages=["snowflake-snowpark-python", 'pandas', 'snowflake-ml-python'])  

In [90]:
sdf = session.table('v1')
sdf.dtypes

[('COL0', 'double'),
 ('COL1', 'double'),
 ('COL2', 'double'),
 ('COL3', 'double'),
 ('COL4', 'double'),
 ('DATE', 'date'),
 ('HH', 'bigint'),
 ('GEO', 'string(16777216)'),
 ('CONTRACT_ID', 'bigint'),
 ('ACTUAL', 'bigint')]

In [91]:
# syntax is, say the columns you want to go in ( the *[] bit below), and say what partitions you want to cut the data up by ( in this case CONTRACT_ID)
sdf_prepped = sdf.select(udtf_model_inf(*['COL0', 'COL1', 'COL2', 'COL3', 'COL4', 'DATE', 'HH', 'GEO', 'CONTRACT_ID', 'ACTUAL']).over(partition_by=['CONTRACT_ID']))

In [92]:
sdf_prepped.show()

---------------------------------------------------------------------
|"DATE"      |"HH"  |"ACTUAL"  |"GEO"  |"CONTRACT_ID"  |"FORECAST"  |
---------------------------------------------------------------------
|2018-01-01  |2     |1.0       |3      |4              |1.0         |
|2018-01-01  |1     |0.0       |0      |4              |0.0         |
|2018-01-01  |1     |1.0       |1      |4              |1.0         |
|2018-01-01  |3     |0.0       |0      |4              |0.0         |
|2018-01-01  |2     |0.0       |0      |4              |0.0         |
|2018-01-01  |1     |1.0       |2      |4              |1.0         |
|2018-01-01  |4     |1.0       |3      |4              |1.0         |
|2018-01-01  |0     |1.0       |4      |4              |1.0         |
|2018-01-01  |2     |1.0       |2      |4              |1.0         |
|2018-01-01  |3     |0.0       |4      |4              |0.0         |
---------------------------------------------------------------------

